In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Unit Testing Partial Derivatives

If you want to check the implementations of a `Component`’s partial derivatives as part of a unit test, you can make use of a custom assert function, `assert_check_partials`.

```{eval-rst}
    .. autofunction:: openmdao.utils.assert_utils.assert_check_partials
        :noindex:
```

In your unit test, after calling `check_partials` on a Component, you can call the `assert_check_partials` function with the returned value from `check_partials`.

## Usage

In the following code, `compute_partials` is intentionally coded incorrectly to demonstrate how `assert_check_partials` can be used to detect this kind of error.

In [ ]:
import numpy as np
import openmdao.api as om
from openmdao.utils.assert_utils import assert_check_partials

class BrokenDerivComp(om.ExplicitComponent):
    def setup(self):
        self.add_input('x1', 3.0)
        self.add_input('x2', 5.0)

        self.add_output('y', 5.5)

    def setup_partials(self):
        self.declare_partials(of='*', wrt='*')

    def compute(self, inputs, outputs):
        """ Compute outputs. """
        outputs['y'] = 3.0 * inputs['x1'] + 4.0 * inputs['x2']

    def compute_partials(self, inputs, partials):
        """Intentionally incorrect derivative."""
        J = partials
        J['y', 'x1'] = np.array([4.0])
        J['y', 'x2'] = np.array([40])


prob = om.Problem()
prob.model.add_subsystem('comp', BrokenDerivComp())

prob.set_solver_print(level=0)

prob.setup()
prob.run_model()

data = prob.check_partials(out_stream=None)
print(data)

try:
    assert_check_partials(data, atol=1.e-6, rtol=1.e-6)
except ValueError as err:
    print(str(err))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal
from openmdao.utils.assert_utils import assert_check_partials
try:
    assert_check_partials(data, atol=1.e-6, rtol=1.e-6)
except ValueError as err:
    err_string = str(err)
    assert err_string.count('Assert Check Partials failed for the following Components') == 1
    assert err_string.count('1e-06') == 2
    assert err_string.count('Component:') == 1
    assert err_string.count('< output > wrt < variable >') == 1
    assert err_string.count('norm') == 2
    assert err_string.count('y wrt x1') == 2
    assert err_string.count('y wrt x2') == 2
    assert err_string.count('abs') == 4
    assert err_string.count('rel') == 4
    assert err_string.count('fwd-fd') == 4
else:
    self.fail('Exception expected.')